Задача - получить бейслайн при поможи DNN (улучшить random forest)

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

In [23]:
df = pd.read_csv('data/titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
y = df['Survived']
X = df.drop('Survived', axis = 1)
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [46]:
def to_float(x):
    def f(x):
        return pd.to_numeric(x, downcast='float')
    return f(x)
    

def prepare_dataframe(df):
    res = pd.get_dummies(df.iloc[:, [1, 3, 4]]).fillna(0)
    res = res.fillna({'Age': df['Age'].median()}).drop(['Sex_female'], axis=1)\
                    .rename(columns={'Sex_male':'Sex'})
    res['Sex'] = res['Sex'] - (res['Sex'].max() - res['Sex'].min()) / 2
    res['Age'] = res['Age'] - (res['Age'].max() - res['Age'].min()) / 2
    res['Pclass'] = res['Pclass'] - (res['Pclass'].max() - res['Pclass'].min()) / 2
    return res.apply(to_float)

res = prepare_dataframe(X)
res['Pclass'].unique()


array([2., 0., 1.], dtype=float32)

In [47]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (3,)),
    tf.keras.layers.Dense(12, activation = 'relu'),
    tf.keras.layers.Dense(12, activation = 'relu'),
    tf.keras.layers.Dense(4, activation = 'relu'),
    tf.keras.layers.Dense(2, activation = 'softmax')
])

model.compile(optimizer= tf.keras.optimizers.SGD(), 
              loss=tf.keras.losses.mean_squared_error)
model.fit(res, y, epochs = 15)

Epoch 1/15
28/28 [==============================] - 0s 3ms/step - loss: 0.2512
Epoch 2/15
28/28 [==============================] - 0s 3ms/step - loss: 0.2504
Epoch 3/15
28/28 [==============================] - 0s 4ms/step - loss: 0.2504
Epoch 4/15
28/28 [==============================] - 0s 4ms/step - loss: 0.2503
Epoch 5/15
28/28 [==============================] - 0s 5ms/step - loss: 0.2503
Epoch 6/15
28/28 [==============================] - ETA: 0s - loss: 0.250 - 0s 6ms/step - loss: 0.2503
Epoch 7/15
28/28 [==============================] - 0s 6ms/step - loss: 0.2503
Epoch 8/15
28/28 [==============================] - 0s 5ms/step - loss: 0.2503
Epoch 9/15
28/28 [==============================] - 0s 4ms/step - loss: 0.2503
Epoch 10/15
28/28 [==============================] - 0s 3ms/step - loss: 0.2503
Epoch 11/15
28/28 [==============================] - 0s 7ms/step - loss: 0.2503
Epoch 12/15
28/28 [==============================] - 0s 5ms/step - loss: 0.2503
Epoch 13/15
28/28 [======

In [71]:
X_t = pd.read_csv('data/titanic/test.csv')
X_t = prepare_dataframe(X_t)

pred = pd.DataFrame({'Survived' : model.predict(X_t)[:,0]})\
                .reset_index().rename(columns={'index':'PassengerId'})
pred['Survived'] = pred['Survived'].apply(lambda x: 0 if x < 0.5 else 1 )  
pred.to_csv('data/titanic/result.csv', index=False)


In [14]:
tf.keras.layers.Dense?